this was for hackathon hosted on zindi for Egyptian universities 
this is its link 
https://zindi.africa/hackathons/umojahack-egypt/

In [ ]:
# from google.colab import drive
# drive.mount('./gdrive')

# !cp "./gdrive/MyDrive/UmojaHackEgypt.zip" .
# !cp "./gdrive/MyDrive/train_tracks.zip" .

# !unzip "/content/train_targets.zip"
# !7z e -pqkptu "/content/UmojaHackEgypt.zip"

In [ ]:
# !pip install catboost
# !pip install pytorch-tabnet
# !unzip  "/content/train_sequences.zip"
# !unzip "/content/train_tracks.zip"
# !unzip "/content/test_sequences.zip"
# !pip install traces


In [ ]:
import pandas as pd
import numpy as np 
from pytorch_tabnet.tab_model import TabNetRegressor
import glob, random
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from catboost import Pool
from sklearn.neighbors import KNeighborsRegressor
import traces


        

In [ ]:
sequences = sorted(glob.glob('train_sequences/*source.csv')) 
targets = [s.replace('sequence', 'target').replace('source', 'target') for s in sequences]
tracks = sorted(glob.glob('train_tracks/*source.csv'))
tracksbench = [t.replace("source","bench") for  t in tracks]
# sequences = sequences +tracks
# targets = targets + tracksbench

In [ ]:
uni = []
for i in range(len(sequences)):
  uni.extend(pd.read_csv(sequences[1]).Movement_Type.unique())
print(pd.Series(uni).unique ())

[2 0]


In [ ]:
pd.Series([3,np.nan,np.nan,3,np.nan,4,7,8]).interpolate(method='linear')

0    3.0
1    3.0
2    3.0
3    3.0
4    3.5
5    4.0
6    7.0
7    8.0
dtype: float64

In [ ]:
def firstindex(series):
  return series[series.notnull()].index[0]

In [ ]:
timesteps = 20
def load_seq(i,timesteps= timesteps):
    source = pd.read_csv(sequences[i])
    target = pd.read_csv(targets[i])
    original_sub = source.copy()
    source.Time = np.round(source.Time).astype("int")
    source = source.drop_duplicates(subset='Latitude')
    preds = pd.DataFrame({
        'Time':range(-timesteps,min(len(original_sub),len(target)))
    })
    # Create traces timeseries for both Latitude and Longitude
    lat_ts = traces.TimeSeries([s for s in source[['Time', 'Latitude']].values])
    lon_ts = traces.TimeSeries([s for s in source[['Time', 'Longitude']].values])
    
    # Resample with linear interpolation and store results
    preds['Latitude'] = [source.Latitude[firstindex(source.Latitude)]]*timesteps + [x[1] for x in lat_ts.sample(sampling_period=1,interpolate='linear', start=0, end=min(len(original_sub),len(target))-1 )]
    preds['Longitude'] = [source.Longitude[firstindex(source.Longitude)]]*timesteps+ [x[1] for x in lon_ts.sample(sampling_period=1,interpolate='linear', start=0, end=min(len(original_sub),len(target))-1)]

    preds['Accuracy'] = original_sub.Accuracy.tolist()[:1] * timesteps + original_sub.Accuracy.tolist()[:min(len(original_sub),len(target))]

    preds['Movement_Type'] = original_sub.Movement_Type.tolist()[:1] * timesteps + original_sub.Movement_Type.tolist()[:min(len(original_sub),len(target))]
    target.columns= [ "Time", "trueLat","trueLong"]
    newTarget = pd.DataFrame({
        'Time':range(-timesteps,min(len(original_sub),len(target)))
    })
    target.drop("Time",axis=1,inplace=True)
    
    newTarget["trueLat"]  = target.trueLat.tolist()[:1] * timesteps + target.trueLat.tolist()[:min(len(original_sub),len(target))]
    newTarget["trueLong"]  = target.trueLong.tolist()[:1] * timesteps + target.trueLong.tolist()[:min(len(original_sub),len(target))]
    newTarget.drop("Time",axis=1,inplace=True)
    preds.drop("Time",axis=1,inplace=True)
    npseq = pd.concat([preds, newTarget],axis=1).to_numpy()

    seqdatax = []
    seqdatay = []
    for i in range(len(npseq) -timesteps ):
      seqdatax.append(npseq[i:i+timesteps,:-2])
      seqdatay.append(npseq[i+timesteps-1,-2:])
    return np.array(seqdatax),np.array(seqdatay)

In [ ]:
datax, datay = load_seq(0,timesteps= timesteps)
for i in range(1,len(targets)):
  datax = np.concatenate((datax,load_seq(i)[0]) )
  datay = np.concatenate((datay,load_seq(i)[1]))


In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        batch_size = input.size(0)
        out = input.view(batch_size,-1)
        return out
class MyTransformer(nn.Module):
  def __init__(self):
    super(MyTransformer,self).__init__()
    self.layers = nn.Sequential(
        nn.TransformerEncoderLayer(5,1,dim_feedforward=128),
        Flatten(),
        nn.Dropout(0.2),
        nn.PReLU(),
        nn.Linear(25,25),
        nn.PReLU(),
        nn.Dropout(0.2),
        nn.Linear(25,2)
    )
  def forward(self,x):
    return self.layers(x)


In [ ]:
"""
actually even if DL was promising at first it couldn't compete with other Tree based models (with stacking )

"""


# model =  MyTransformer().to(device)

# optimizer = optim.Adam(model.parameters(),lr=0.001,weight_decay=0.01)

# n_samples = len(dataxtrain)
# batch_size= 128
# better_batch_size = ceil(n_samples, ceil(n_samples, batch_size))
# epochs= 50
# for e in range(epochs):
#   model.train()
#   trainloss = []
#   for i in range(ceil(n_samples, better_batch_size)): 
#     batchx = dataxtrain[i*better_batch_size :(i+1)*better_batch_size]
#     batchy = dataytrain[i*better_batch_size :(i+1)*better_batch_size]
#     model.zero_grad()
#     batchy = batchy * 1000
#     batchy = torch.Tensor(batchy).to(device)
#     batchx = torch.Tensor(batchx).to(device)
#     preds = model(batchx)
#     loss = F.mse_loss(preds,batchy)
#     loss.backward()
#     trainloss.append(loss.item())
#     optimizer.step()
  
#   model.eval()
#   valLosses = []
#   for i in range(ceil(len(dataxvalid), better_batch_size)): 
#     batchx = torch.Tensor(dataxvalid[i*better_batch_size:(i+1)*better_batch_size]).to(device)
#     valpreds = model(batchx)
#     valpreds = valpreds/ 1000
#     lossStep = np.sqrt(mean_squared_error(datayvalid[i*better_batch_size:(i+1)*better_batch_size],valpreds.cpu().detach().numpy()))
#     valLosses.append(lossStep)
#   print(e,np.mean(valLosses),"validation",np.mean(trainloss),"train")


 0.0009185077056372776 

In [ ]:
ss = pd.read_csv("SampleSubmission.csv")

In [ ]:

ss["cord"] = ss["Row_ID"].apply(lambda x : x.split(" X ")[-1])
ss["seq_id"] = ss["Row_ID"].apply(lambda x : x.split(" X ")[0])
ss["time"] = ss["Row_ID"].apply(lambda x : int(x.split(" X ")[1][4:]))


In [ ]:
ss = ss.sort_values(["seq_id","time"])

In [ ]:
datax[0,0]

array([-2.00000000e+01, -1.46009872e-03,  4.29911647e-04,  1.64640007e+01,
        2.00000000e+00])

In [ ]:
def load_seq_test(seq_id,timesteps= timesteps):
  
    source = pd.read_csv(f'test_sequences/{seq_id}_source.csv')
    source.Time = np.round(source.Time).astype("int")
    original_sub = source.copy()
    
    source = source.drop_duplicates(subset='Latitude')
    preds = pd.DataFrame({
        'Time':range(-timesteps,len(original_sub))
    })
    # Create traces timeseries for both Latitude and Longitude
    lat_ts = traces.TimeSeries([s for s in source[['Time', 'Latitude']].values])
    lon_ts = traces.TimeSeries([s for s in source[['Time', 'Longitude']].values])
    
    # Resample with linear interpolation and store results
    preds['Latitude'] = [source.Latitude[firstindex(source.Latitude)]]*timesteps + [x[1] for x in lat_ts.sample(sampling_period=1,interpolate='linear', start=0, end=len(original_sub)-1 )]
    preds['Longitude'] = [source.Longitude[firstindex(source.Longitude)]]*timesteps+ [x[1] for x in lon_ts.sample(sampling_period=1,interpolate='linear', start=0, end=len(original_sub)-1)]

    preds['Accuracy'] = original_sub.Accuracy.tolist()[:1] * timesteps + original_sub.Accuracy.tolist()

    preds['Movement_Type'] = original_sub.Movement_Type.tolist()[:1] * timesteps + original_sub.Movement_Type.tolist()
    preds.drop("Time",axis=1,inplace=True)
    npseq = preds.to_numpy()
    seqdatax = []
  
    for i in range(len(npseq) -timesteps ):
      seqdatax.append(npseq[i:i+timesteps].tolist())
    return np.array(seqdatax)


In [ ]:
ss.to_csv("sub_weight_decay01_lr001.csv",index=False)

In [ ]:
dataxtrain ,dataxvalid, dataytrain,datayvalid = train_test_split(datax,datay,test_size=0.3,shuffle=True)


tempx = dataxtrain.copy()
tempy = dataytrain.copy()

dataxtrain = tempx[:int(0.5*len(tempx))]
dataytrain = tempy[:int(0.5*len(tempx))]
dataxtrain1 = tempx[int(0.5*len(tempx)):]
dataytrain1 = tempy[int(0.5*len(tempx)):]


halfxtrain = datax[:int(0.5*len(datax))]
halfytrain = datay[:int(0.5*len(datax))]
halfxvalid = datax[int(0.5*len(datax)):]
halfyvalid = datay[int(0.5*len(datax)):]



In [ ]:
np.sqrt(mean_squared_error(dataytrain,xgb.predict(dataxtrain.reshape(len(dataxtrain),-1))))

0.00017181069156477124

In [ ]:
xgb = XGBRegressor(objective="reg:squarederror",max_depth =9, learning_rate=0.03  ,n_estimators=450,random_seed=32)

xgb = MultiOutputRegressor(xgb)
cat = CatBoostRegressor(iterations= 1000,verbose=200,random_seed=23,learning_rate=0.03,depth=6,l2_leaf_reg=0.3) 
cat = MultiOutputRegressor(cat)

dt = DecisionTreeRegressor()
dt = MultiOutputRegressor(dt)


xgb.fit(dataxtrain.reshape(len(dataxtrain),-1),dataytrain)
cat.fit(dataxtrain.reshape(len(dataxtrain),-1),dataytrain)

dt.fit(dataxtrain.reshape(len(dataxtrain),-1),dataytrain)



/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


0:	learn: 0.0009944	total: 29.7ms	remaining: 29.6s
200:	learn: 0.0001465	total: 5.1s	remaining: 20.3s
400:	learn: 0.0001226	total: 10s	remaining: 15s
600:	learn: 0.0001111	total: 15.1s	remaining: 10s
800:	learn: 0.0001038	total: 20.1s	remaining: 5s
999:	learn: 0.0000985	total: 25.2s	remaining: 0us
0:	learn: 0.0009853	total: 26.8ms	remaining: 26.8s
200:	learn: 0.0001350	total: 5.07s	remaining: 20.2s
400:	learn: 0.0001124	total: 10s	remaining: 15s
600:	learn: 0.0001021	total: 15s	remaining: 9.94s
800:	learn: 0.0000958	total: 20s	remaining: 4.97s
999:	learn: 0.0000911	total: 25s	remaining: 0us


MultiOutputRegressor(estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                     criterion='mse',
                                                     max_depth=None,
                                                     max_features=None,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort='deprecated',
                                                     random_state=None,
                                                     splitter='best'),
                     n_jobs=None)

0.00010791249808818972

In [ ]:

predxgb =  xgb.predict(dataxtrain1.reshape(len(dataxtrain1),-1))
predcat =  cat.predict(dataxtrain1.reshape(len(dataxtrain1),-1))
preddt =  dt.predict(dataxtrain1.reshape(len(dataxtrain1),-1))


predxgbvalid =  xgb.predict(dataxvalid.reshape(len(dataxvalid),-1))
predcatvalid =  cat.predict(dataxvalid.reshape(len(dataxvalid),-1))

preddtvalid =  dt.predict(dataxvalid.reshape(len(dataxvalid),-1))

trainpreds = np.concatenate((dataxtrain1.reshape(len(dataxtrain1),-1),predxgb,predcat,preddt),axis=1)
validpreds = np.concatenate((dataxvalid.reshape(len(dataxvalid),-1),predxgbvalid,predcatvalid,preddtvalid),axis=1)


xgbfinal = CatBoostRegressor(iterations= 800,verbose=200,learning_rate=0.03,depth=6,l2_leaf_reg=0.3) 
xgbfinal = MultiOutputRegressor(xgbfinal)
xgbfinal.fit(trainpreds,dataytrain1)

finalpreds = xgbfinal.predict(validpreds)

np.sqrt(mean_squared_error(datayvalid,finalpreds))




0:	learn: 0.0009894	total: 34.5ms	remaining: 27.5s
200:	learn: 0.0001168	total: 5.54s	remaining: 16.5s
400:	learn: 0.0001056	total: 10.8s	remaining: 10.8s
600:	learn: 0.0000988	total: 16.2s	remaining: 5.35s
799:	learn: 0.0000939	total: 21.5s	remaining: 0us
0:	learn: 0.0009877	total: 27.4ms	remaining: 21.9s
200:	learn: 0.0001066	total: 5.42s	remaining: 16.2s
400:	learn: 0.0000965	total: 10.8s	remaining: 10.7s
600:	learn: 0.0000906	total: 16s	remaining: 5.31s
799:	learn: 0.0000865	total: 21.4s	remaining: 0us


9.766775778358494e-05

In [ ]:

# predxgb =  xgb.predict(dataxtrain1.reshape(len(dataxtrain1),-1))
# predcat =  cat.predict(dataxtrain1.reshape(len(dataxtrain1),-1))
# # predtab =  tab.predict(dataxtrain1.reshape(len(dataxtrain1),-1))
# # predrf =  rf.predict(dataxtrain1.reshape(len(dataxtrain1),-1))
# preddt =  dt.predict(dataxtrain1.reshape(len(dataxtrain1),-1))
# # predknn =  knn.predict(dataxtrain1.reshape(len(dataxtrain1),-1))


# predxgbvalid =  xgb.predict(dataxvalid.reshape(len(dataxvalid),-1))
# predcatvalid =  cat.predict(dataxvalid.reshape(len(dataxvalid),-1))
# # predtabvalid =  tab.predict(dataxvalid.reshape(len(dataxvalid),-1))
# # predrfvalid =  rf.predict(dataxvalid.reshape(len(dataxvalid),-1))
# preddtvalid =  dt.predict(dataxvalid.reshape(len(dataxvalid),-1))
# # predknnvalid =  knn.predict(dataxvalid.reshape(len(dataxvalid),-1))

# trainpreds = np.concatenate((dataxtrain1.reshape(len(dataxtrain1),-1),predxgb,predcat,preddt),axis=1)
# validpreds = np.concatenate((dataxvalid.reshape(len(dataxvalid),-1),predxgbvalid,predcatvalid,preddtvalid),axis=1)


xgb1final.fit(trainpreds,dataytrain1)
xgbfinal.fit(trainpreds,dataytrain1)

finalpreds1 = xgb1final.predict(validpreds)
finalpreds = xgbfinal.predict(validpreds)

lastTrainx = np.concatenate((finalpreds,finalpreds1),axis=1)
lr = DecisionTreeRegressor() 
lr = MultiOutputRegressor(lr)
lr.fit(lastTrainx,datayvalid)
finalfinalpreds = lr.predict(lastTrainx)


np.sqrt(mean_squared_error(datayvalid,finalfinalpreds))



0.00010733667616236996

In [ ]:
xgb
xgb1.fit(dataxtrain.reshape(dataxtrain.shape[0],-1),dataytrain)
preds = xgb1.predict(dataxvalid.reshape(dataxvalid.shape[0],-1))
np.sqrt(mean_squared_error(datayvalid,preds))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


0.00013933667616236996

0.0003463275090063775

In [ ]:
32 ** 10

In [ ]:

for i in ss["seq_id"].unique():
  data = load_seq_test(i,timesteps=timesteps)
  
  predxgbvalid =  xgb.predict(data.reshape(len(data),-1))
  predcatvalid =  cat.predict(data.reshape(len(data),-1))
  preddtvalid =  dt.predict(data.reshape(len(data),-1))

  validpreds = np.concatenate((data.reshape(len(data),-1),predxgbvalid,predcatvalid,preddtvalid),axis=1)

  finalpreds = xgbfinal.predict(validpreds)


  # xgbfinal = XGBRegressor(objective="reg:squarederror",max_depth =6, learning_rate=0.03  ,n_estimators=120,random_seed=32)
  
  # xgbfinal = MultiOutputRegressor(xgbfinal)
  # xgbfinal.fit(trainpreds,dataytrain1)

  ss.loc[(ss.seq_id == i)&(ss.cord=='Longitude'), 'Prediction'] = finalpreds[:,1] 
  ss.loc[(ss.seq_id == i)&(ss.cord=='Latitude'), 'Prediction'] = finalpreds[:,0] 
ss.to_csv("stacking.csv",index=False)
